In [1]:
# !pip install rdflib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\rivasa\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [37]:
import pandas as pd
import json

from ipycytoscape import *
import ipywidgets as widgets
from IPython.display import clear_output
import os
# import bqplot
# from bqplot import pyplot as bqplt
# from bqplot import Tooltip

# Visualizing Cluster generated by SemEP

In [38]:
path = 'semep-solver-master/graph_country_category-15h-49m-46s-Clusters/'
entries = os.listdir(path)

## Select the Cluster to Plot

In [43]:
dropdown_cluster = widgets.Dropdown(options = {}, description='Cluster:')
dropdown_cluster.options=entries
display(dropdown_cluster)

Dropdown(description='Cluster:', options=('cluster-0.txt', 'cluster-1.txt', 'cluster-10.txt', 'cluster-11.txt'…

In [40]:
def load_cluster(name):
    cls = pd.read_csv(path + name, delimiter="\t", header=None)
    cls.columns = ['Country', 'category', 'Indicator', 'Cost']
    return cls

def create_json_to_cytoscape(name): # entity_type
    rdf_graph = load_cluster(name)
    graph_json = dict()
    graph_json['nodes'] = []
    graph_json['edges'] = []
    drug_id = dict()
    id_x = 0
    for i in range(rdf_graph.shape[0]):
        subj = rdf_graph.iloc[i]['Country']
        obj = rdf_graph.iloc[i]['category']
        prop = rdf_graph.iloc[i]['Indicator']
        edge = dict()
        edge['data'] = dict()

        if subj in drug_id.keys():
            edge['data']['id'] = id_x
            edge['data']['source'] = drug_id[subj]
            edge['data']['Label'] = prop
            id_x += 1
        else:
            node = dict()
            node['data'] = dict()
            drug_id[subj] = id_x
            node['data']['id'] = id_x
            node['data']['name'] = subj
            node['data']['type'] = 'Country' # entity_type[subj]
            edge['data']['id'] = id_x + 1
            edge['data']['source'] = id_x
            edge['data']['Label'] = prop
            graph_json['nodes'].append(node)
            id_x += 2
        if obj in drug_id.keys():
            edge['data']['target'] = drug_id[obj]
        else:
            node = dict()
            node['data'] = dict()
            drug_id[obj] = id_x
            node['data']['id'] = id_x
            node['data']['name'] = obj
            node['data']['type'] = 'category' # entity_type[obj]
            edge['data']['target'] = id_x
            graph_json['nodes'].append(node)
            id_x += 1
#             if obj == k:
#                 node['classes'] = 'red'  # Single class

        graph_json['edges'].append(edge)

    return graph_json

def create_graph_cytoscape(name):
    middle_vertex = create_json_to_cytoscape(name)
    # load a style dictionary
    with open("styles.json") as fi:
        s = json.load(fi)
    # Create the cytoscape graph widget
    cytoscapeobj = CytoscapeWidget()
    cytoscapeobj.graph.add_graph_from_json(middle_vertex, directed=True, multiple_edges=True)  # , directed=True, input_data['elements']
    
    cytoscapeobj.set_style(s)
    cytoscapeobj.set_layout(name='dagre', animate=True, nodeSpacing = 5)  # concentric,  breadthfirst, fcose, dagre, grid
    return cytoscapeobj

## Visualize the cluster selected

In [42]:
button = widgets.Button(description="Visualize")
output_2 = widgets.Output()
display(button, output_2)

def on_button_clicked(tfry):
    with output_2:
        clear_output(wait=True)
#         graph_json = create_json_to_cytoscape(cls)
        display(create_graph_cytoscape(dropdown_cluster.value))

button.on_click(on_button_clicked)

Button(description='Visualize', style=ButtonStyle())

Output()

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()